In [112]:
! pip install PyLD
from pyld import jsonld
from copy import deepcopy
import json

In [103]:
def get_activities(applet_expanded):
    activities = [a['@id'] for a in applet_expanded[0]['https://schema.repronim.org/order'][0]['@list']]

    activities_expanded = {a: jsonld.expand(a) for a in activities}
    return activities_expanded

def get_items(activities_expanded):
    items_expanded = {}
    for a in activities_expanded.keys():
        for i in activities_expanded[a][0]['https://schema.repronim.org/order'][0]['@list']:
            items_expanded[i['@id']] = jsonld.expand(i['@id'])
    return items_expanded

In [104]:
def check_for_unexpanded_value_constraints(item_exp):
    vc = item_exp[0]
    if 'https://schema.repronim.org/valueconstraints' in vc.keys():
        vc = vc['https://schema.repronim.org/valueconstraints'][0]
        if isinstance(vc, dict):
            if "@id" in vc.keys():
                return True

    return False

In [105]:
def expand_value_constraints(original_items_expanded):
    items_expanded = deepcopy(original_items_expanded)
    for item, item_exp in original_items_expanded.items():
        # check if we need to expand valueConstraints
        vc = item_exp[0]
        if 'https://schema.repronim.org/valueconstraints' in vc.keys():
            if check_for_unexpanded_value_constraints(item_exp):
                vc = jsonld.expand(item_exp[0]['https://schema.repronim.org/valueconstraints'][0]['@id'])
                items_expanded[item][0]['https://schema.repronim.org/valueconstraints'][0] = vc
        else:
            multipart_activities = get_activities(item_exp)
            items_expanded.update(multipart_activities)
    return items_expanded

In [106]:
def expand_full(appletURL):
    applet_expanded = jsonld.expand(appletURL)
    
    activities_expanded = get_activities(applet_expanded)
    
    items_expanded = get_items(activities_expanded)
    expItems1 = expand_value_constraints(items_expanded)
    
    # re-expand the items in case any multiparters were added
    expItems2 = expand_value_constraints(expItems1)
    
    return dict(activities = activities_expanded, items = items_expanded, applet = applet_expanded)

In [114]:
appletURL = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activity-sets/voice-pilot/voice_pilot_schema.jsonld'

fully_expanded_voice = expand_full(appletURL)

In [117]:
# json.dumps(fully_expanded_voice)

In [109]:
hbn_applet = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activity-sets/ema-hbn/ema-hbn_schema.jsonld'
fully_expanded_hbn = expand_full(hbn_applet)

In [116]:
json.dumps(fully_expanded_hbn)

'{"activities": {"https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNMorning/ema_morning_schema.jsonld": [{"@id": "https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNMorning/ema_morning_schema", "@type": ["https://raw.githubusercontent.com/ReproNim/schema-standardization/master/schemas/Activity.jsonld"], "http://schema.repronim.org/preamble": [{"@language": "en", "@value": ""}], "http://schema.org/description": [{"@language": "en", "@value": "Morning Questions"}], "http://schema.org/schemaVersion": [{"@language": "en", "@value": "0.0.1"}], "http://schema.org/version": [{"@language": "en", "@value": "0.0.1"}], "https://schema.repronim.org/scoringLogic": [], "http://www.w3.org/2004/02/skos/core#altLabel": [{"@language": "en", "@value": "ema_morning_schema"}], "http://www.w3.org/2004/02/skos/core#prefLabel": [{"@language": "en", "@value": "EMA: Morning"}], "https://schema.repronim.org/order": [{"@list": [{"@id":